<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/two_tower_BERTmodel_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install scikit-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 41.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you hav

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7f5c17d8-3c82-f9c3-4de6-9a053e98618a)


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv("./job_PII_diagnosis_2.csv", error_bad_lines=False)
df.head(2)

<ipython-input-16-66fd34e864f1>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("./job_PII_diagnosis_2.csv", error_bad_lines=False)


,prompt,related_to_job
0,"I am doing Marketing Intern at US, NY, New Yor...",1
1,I am doing in the industry of Success What we ...,1


In [17]:
# data split
X_train, X_test, y_train, y_test = train_test_split(df['prompt'], df['related_to_job'], test_size=0.2, random_state=42)

In [18]:
#allow us to get the hidden layer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

In [19]:
X_train_list = X_train.apply(lambda x: x.replace('\xa0', ' ')).tolist()
X_val_list = X_test.apply(lambda x: x.replace('\xa0', ' ')).tolist()

# train
bert_train_tokenized = bert_tokenizer(X_train_list,
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='tf')
bert_train_inputs = [bert_train_tokenized.input_ids,
    bert_train_tokenized.token_type_ids,
    bert_train_tokenized.attention_mask]
bert_train_labels = np.array(y_train)

# valdation
bert_val_tokenized = bert_tokenizer(X_val_list,
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='tf')
bert_val_inputs = [bert_val_tokenized.input_ids,
    bert_val_tokenized.token_type_ids,
    bert_val_tokenized.attention_mask]
bert_val_labels = np.array(y_test)

In [20]:
print(X_train_list[0])
print(bert_train_labels[0])

 nan, Overview: Looking for an experienced hands-on Java developer with at least 7-10 years’ experience to join the team. We are building Stress testing sol, and the benefits: negotiable
1


In [21]:
text = X_train_list[0]
encoded_input = bert_tokenizer.encode_plus(text, return_tensors='pt')
encoded_input['input_ids'].size(1)

47

In [23]:
#12 layers of transformer
#A drop out layer + dense layer with 100 hidden layer size on top + final layer with sigmoid as activation function
def create_bert_cls_model(bert_base_model,
                          max_sequence_length=128,
                          hidden_size = 100,
                          dropout=0.3,
                          learning_rate=0.0001): #0.0005
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """

    bert_base_model.trainable = True #True

    #input layers of BERT, shape (batch, max_sequence_length), model will be fit with bert_train_tokenized
    input_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name='input_ids')
    token_type_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name='token_type_ids')
    attention_mask = Input(shape=(max_sequence_length,), dtype=tf.int32, name='attention_mask')

    inputs = [input_ids, token_type_ids, attention_mask]

    #BERT output, last_hidden_state shape (batch, max_sequence_length, embedding dimensions)
    bert_output = bert_base_model(input_ids=input_ids,
                                  token_type_ids=token_type_ids,
                                  attention_mask=attention_mask)

    #Extract the CLS token's output, the embedding representation of first token of every sentence, shape(batch, embedding dimensions)
    cls_token_output = bert_output[0][:, 0, :] # CLS token output from the last layer

    #Add a dropout layer
    x = Dropout(dropout)(cls_token_output)

    #Add a fully connected layer for classification
    x = Dense(hidden_size, activation='relu')(x)

    #Final output layer for classification, assuming it's binary task
    output = Dense(1, activation='sigmoid')(x)

    #Model complie
    classification_model = Model(inputs=inputs, outputs=output)
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss='binary_crossentropy',
                                 metrics=['accuracy'])
    return classification_model

In [24]:
bert_cls_model_classification = create_bert_cls_model(bert_model)

In [25]:
history_cls_bert= bert_cls_model_classification.fit(bert_train_inputs,
                                                    bert_train_labels,
                                                    epochs=1, #2
                                                    batch_size=16, #8
                                                    validation_data=(bert_val_inputs, bert_val_labels))

# 56/567 accuracy 0.95 data examples 4000 batch 8 lr 0.00005
# 12/100 accuracy 0.92 data examples 2000 batch 16 lr 0.0001

100/100 [==============================] - 1520s 15s/step - loss: 0.0450 - accuracy: 0.9856 - val_loss: 0.1692 - val_accuracy: 0.9800


In [35]:
bert_cls_model_classification.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',   

In [67]:
bert_cls_model_classification.save('./two_tower_bert.h5')

In [70]:
from tensorflow.keras.models import load_model

custom_objects = {'TFBertModel': TFBertModel}
model = load_model('./two_tower_bert.h5', custom_objects=custom_objects)

BERT as the embedding for the vector database

In [71]:
def generate_embedding(text, model):
  bert_embedding_model = Model(inputs=[model.input],
                               outputs=model.get_layer('tf.__operators__.getitem_1').output)
  bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)

text = X_train_list[2]
embeddings = generate_embedding(text, model)
embeddings.shape

1/1 [==============================] - 7s 7s/step


(1, 768)

In [ ]:
!pip install faiss-gpu transformers torch --quiet

In [48]:
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings
nlist = 50  # Number of clusters for quantization, adjust based on your dataset size

# Using an IndexFlatL2 for simplicity, which is a basic but exact method for similarity search
index = faiss.IndexFlatL2(dimension)

# For larger datasets, consider using an IVF index to speed up search at the cost of slight accuracy loss
# quantizer = faiss.IndexFlatL2(dimension)
# index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
# index.train(embeddings) # You need to train the index with some embeddings if using IVF

In [58]:
def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings


# Add embeddings to the index
index.add(embedding_vectordatabase(embeddings))

In [ ]:
"""
Business Analyst - CRM and the salary range is negotiable Our passion for improving quality of life through geography is at the heart of everything we do.
Esri’s geographic information system (GIS) technolog

"""

In [66]:
query_text = "My name is Ryan and I have a headache"
query_embedding = generate_embedding(query_text)
query_embedding = embedding_vectordatabase(query_embedding)

# Search the index
k = 5
distances, indices = index.search(query_embedding, k)

print("Indices of closest embeddings:", indices)
print("Distances:", distances)

1/1 [==============================] - 6s 6s/step
Indices of closest embeddings: [[ 1  2  0 -1 -1]]
Distances: [[9.7718237e+02 9.7718237e+02 9.7718237e+02 3.4028235e+38 3.4028235e+38]]


In [64]:
num_vectors = index.ntotal
print(f"The index contains {num_vectors} vectors.")

The index contains 3 vectors.


In [ ]:
#save the model
model.save('path_to_my_model.h5')

In [75]:
from google.colab import files

files.download('./two_tower_bert.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>